In [2]:
from CVPreClass import CVPreClass as cvpre

In [3]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse

In [4]:
import tensorflow as tf
from tensorflow import keras 
import os

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

In [6]:
keras.__version__
tf.__version__

'2.1.0'

In [ ]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.6  # 0.6 sometimes works better for folks
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)

print(physical_devices)

In [7]:
def build(width, height, depth, classes):
# initialize the model along with the input shape to be
# "channels last"
    model = Sequential()
    inputShape = (height, width, depth)

    # if we are using "channels first", update the input shape
    if K.image_data_format() == "channels_first":
        inputShape = (depth, height, width)

        # define the first (and only) CONV => RELU layer
    model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
    model.add(Activation("relu"))

    # softmax classifier
    model.add(Flatten())
    model.add(Dense(classes))
    model.add(Activation("softmax"))

        # return the constructed network architecture
    return model

In [8]:
trainX,trainY = cvpre.load_images('dataset/val_spl/')
testX,testY = cvpre.load_images('dataset/test_spl/')

/home/digitup1/projects/Cars_Classifiers/CVPreClass/CVPreClass.py:120: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return (np.array(data), np.array(labels))


In [ ]:
trainX = cvpre.resize_ignore_aspect(trainX,200,200)
testX = cvpre.resize_ignore_aspect(testX,200,200)

In [ ]:
trainX = cvpre.keras_dim_rearrange(trainX)
testX = cvpre.keras_dim_rearrange(testX)

In [ ]:
trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

In [ ]:
# convert the labels from integers to vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

In [ ]:
print("[INFO] compiling model...")
opt = SGD(lr=0.005)
model = build(width=200, height=200, depth=3, classes=196)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

In [ ]:
trainX.shape

In [ ]:
# train the network
print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_data=(testX, testY),batch_size=32, epochs=100, verbose=1)